In [ ]:
import os
from os.path import exists
from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from shutil import copyfile
import torchvision
from PIL import Image
import io
import random
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
from torch.utils.data import TensorDataset, ConcatDataset, DataLoader
import pandas as pd
from torchvision.transforms import ToTensor, Lambda
from torchvision.io import read_image
import matplotlib.pyplot as plt
import torch.optim as optim
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from skorch import NeuralNetClassifier
import sklearn

In [ ]:
class CustomImageDataset():
    def __init__(self, annotations_file, img_dir,
            transform=torchvision.transforms.Compose([
                transforms.ToPILImage(),
                transforms.Grayscale(num_output_channels=3),
                transforms.ToTensor(),
                #Resizing all of the imagesin the dataset to (240,240)
                transforms.Resize((240,240)),
                #Normalization using the mean and variance over the whole dataset
                transforms.Normalize((0.5352, 0.5352, 0.5352),(0.3009, 0.3009, 0.3009))]),
                 #Lambda function is used for turning the integer into a tensor
            target_transform=Lambda(lambda y: torch.zeros(5, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))):
        self.img_labels = pd.read_csv(annotations_file)   #Dataaset labels
        self.img_dir = img_dir      #Dataset image path     
        self.transform = transform   
        self.target_transform = target_transform
        

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        if(os.path.exists(img_path)):
            image = read_image(img_path)
            label = torch.tensor(int(self.img_labels.iloc[idx, 1]))
            if self.transform:
                image = self.transform(image)
            image.shape
            return image, label

In [ ]:
# os.chdir("/content/drive/MyDrive/COMP-6721(AI_project)/Submission");
# path = "C:\Concordia_Program\AI\project_part1\Sample images"
# model = torch.load('C:\Concordia_Program\AI\project_phase1\ourmodel.pth')
# model.load_state_dict(torch.load("/content/drive/MyDrive/COMP-6721(AI_project)/Submission/ourmodel.pth"))


In [ ]:
batch_size = 32
validation_split = .2
shuffle_dataset = True
random_seed= 42   #Preventing from random events in each run
sample_dataset1=CustomImageDataset(annotations_file=r'C:\Concordia_Program\AI\project_part2\female_test.csv', img_dir=r'C:\Concordia_Program\AI\project_part2\female_test')
sample_dataset2=CustomImageDataset(annotations_file=r'C:\Concordia_Program\AI\project_part2\male_test.csv', img_dir=r'C:\Concordia_Program\AI\project_part2\male_test')
sample_dataset3 = CustomImageDataset(annotations_file=r'C:\Concordia_Program\AI\project_part1\sample-label.csv', img_dir=r'C:\Concordia_Program\AI\project_part1\Sample images')

test_loader1 = torch.utils.data.DataLoader(sample_dataset1, batch_size,shuffle=False)
test_loader2 = torch.utils.data.DataLoader(sample_dataset2, batch_size,shuffle=False)
test_loader3 = torch.utils.data.DataLoader(sample_dataset3, batch_size,shuffle=False)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1), #First layer
            nn.BatchNorm2d(32),                 #Batch normalization
            nn.LeakyReLU(inplace=True),         #Leaky ReLU activation function
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1), #Second layer
            nn.BatchNorm2d(32),                #Batch normalization
            nn.LeakyReLU(inplace=True),        #Leaky ReLU activation function
            nn.MaxPool2d(kernel_size=2, stride=2),     #Max pooling
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1), #Third layer
            nn.BatchNorm2d(64),              #Batch normalization
            nn.LeakyReLU(inplace=True),      #Leaky ReLU activation function
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1), #Fourth layer
            nn.BatchNorm2d(64),             #Batch normalization
            nn.LeakyReLU(inplace=True),     #Leaky ReLU activation function
            nn.MaxPool2d(kernel_size=2, stride=2),    #Max pooling
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1), #Fifth layer
            nn.BatchNorm2d(64),            #Batch normalization
            nn.LeakyReLU(inplace=True),    #Leaky ReLU activation function
            nn.MaxPool2d(kernel_size=2, stride=2)    #Max pooling
          )

        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),   #0.1 dropout rate
            nn.Linear(57600, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 5) #outputs = number of classes
          )

    def forward(self, x):
        # conv layers
        x = self.conv_layer(x)
        # flatten
        x = x.view(x.size(0), -1)
        #fc layer
        x = self.fc_layer(x)
        
        return x

In [ ]:
import torchvision.models as models
model = CNN()
#r'C:\new downloads\ourmodel.pth'
model.load_state_dict(torch.load(r'C:\new downloads\ourmodel.pth'))
model.eval()


CNN(
  (conv_layer): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01, inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01, inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): LeakyReLU(negative_slope=0.01, inplace=True)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): LeakyReLU(negative_slope=0.01, inplace=True)
    (13):

In [ ]:
#model.eval()
with torch.no_grad():
  correct = 0
  total = 0

for images, labels in test_loader1:
  outputs = model(images)
  _, predicted = torch.max(outputs.data, 1) #Computing the prediction of the model 
  total += labels.size(0)
  correct += (predicted == labels).sum().item() #Computing the number of correct predicted items
print('Test Accuracy of the model on {} female test images : {} %'.format(len(sample_dataset1),(correct / total) * 100))

for images, labels in test_loader2:
  outputs = model(images)
  _, predicted = torch.max(outputs.data, 1) #Computing the prediction of the model 
  total += labels.size(0)
  correct += (predicted == labels).sum().item() #Computing the number of correct predicted items
print('Test Accuracy of the model on {} male test images : {} %'.format(len(sample_dataset2),(correct / total) * 100))

for images, labels in test_loader3:
  outputs = model(images)
  _, predicted = torch.max(outputs.data, 1) #Computing the prediction of the model 
  total += labels.size(0)
  correct += (predicted == labels).sum().item() #Computing the number of correct predicted items
print('Test Accuracy of the model on {} mixed-gender test images : {} %'.format(len(sample_dataset3),(correct / total) * 100))



Test Accuracy of the model on 249 female test images : 61.44578313253012 %
Test Accuracy of the model on 247 male test images : 58.06451612903226 %
Test Accuracy of the model on 100 mixed-gender test images : 59.73154362416108 %


## Classifying a single image

In [ ]:
from PIL import Image
classes = ('Cloth mask', 'N95 and FFP2', 'N95 and FFP2 with valve', 'Surgical mask','No mask')
loader = torchvision.transforms.Compose([
                #Resizing all of the imagesin the dataset to (240,240)
                transforms.Resize((240,240)),
                transforms.ToTensor(),
                #Normalization using the mean and variance over the whole dataset
                transforms.Normalize((0.5352, 0.5352, 0.5352),(0.3009, 0.3009, 0.3009))])
def classify(model, loader, path):
    """load image, returns cuda tensor"""
    model = model.eval()
    image = Image.open(path)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    output = model(image)
    _, predicted = torch.max(output.data, 1)
    print(predicted.item())

print(classify(model= model, loader= loader, path = r'C:\Concordia_Program\AI\project_part2\Male_dataset\0\#cloth mask#_5.jpg') )


0
None
